In [1]:
from collections import OrderedDict
from pprint import pprint


class Node:
    def __init__(self, name):
        self.name = name
    
    def __repr__(self):
        return "Node({})".format(self.name)


class Link:
    def __init__(self, target, weight):
        self.target = target
        self.weight = weight
    
    def __hash__(self):
        return hash((self.target, self.weight))
    
    def __eq__(self, other):
        if self.target == other.target and self.weight == other.weight:
            return True
        return False
    
    def __repr__(self):
        return "Link({}, {:d})".format(self.target, self.weight)


class Network:
    def __init__(self):
        self._nodes = {}
    
    def add_node(self, node):
        if node not in self._nodes:
            self._nodes[node] = set()
    
    def link_nodes(self, node_a, node_b, weight):
        if node_a in self._nodes and node_b in self._nodes and node_a != node_b:
            a = self._nodes[node_a]
            b = self._nodes[node_b]
            a.add(Link(node_b, weight))
            b.add(Link(node_a, weight))
    
    def get_node_links(self, node):
        if node in self._nodes:
            return sorted(self._nodes[node], key=lambda n: n.weight)
    
    def display(self):
        pprint(self._nodes)

In [2]:
# Network from: https://brilliant.org/wiki/dijkstras-short-path-finder/
node_names = ['A', 'B', 'C', 'D', 'E', 'F', 'Home', 'School']
links = [(0,3,3), (0,6,3), (1,6,2), (1,3,1), (1,4,6),
         (2,6,5), (2,4,2), (3,5,4), (4,7,4), (4,5,2),
         (5,7,2)]

# Create nodes & network
nodes = []
net = Network()

for i in range(len(node_names)):
    node = Node(node_names[i])
    net.add_node(node)
    nodes.append(node)

for a, b, weight in links:
    net.link_nodes(nodes[a], nodes[b], weight)

net.display()

{Node(A): {Link(Node(Home), 3), Link(Node(D), 3)},
 Node(B): {Link(Node(D), 1), Link(Node(E), 6), Link(Node(Home), 2)},
 Node(C): {Link(Node(E), 2), Link(Node(Home), 5)},
 Node(D): {Link(Node(A), 3), Link(Node(B), 1), Link(Node(F), 4)},
 Node(E): {Link(Node(B), 6),
           Link(Node(C), 2),
           Link(Node(School), 4),
           Link(Node(F), 2)},
 Node(F): {Link(Node(E), 2), Link(Node(D), 4), Link(Node(School), 2)},
 Node(Home): {Link(Node(A), 3), Link(Node(C), 5), Link(Node(B), 2)},
 Node(School): {Link(Node(F), 2), Link(Node(E), 4)}}


In [3]:
class DijkstraSPF:
    _WEIGHT = 0
    _NODE = 1
    
    def __init__(self, network):
        self._network = network
    
    def get_tree(self, start_node):
        visited = set()
        tree = {}
        to_visit = OrderedDict({start_node: True})
        
        while len(to_visit) > 0:
            current_node = to_visit.popitem(0)[0]
            visited.add(current_node)
            
            if current_node not in tree:
                tree[current_node] = (0, None)
            links = self._network.get_node_links(current_node)
            
            for link in links:
                if link.target not in visited:
                    current_weight = tree[current_node][DijkstraSPF._WEIGHT]
                    total_weight = current_weight + link.weight

                    if link.target not in tree or total_weight < tree[link.target][DijkstraSPF._WEIGHT]:
                        tree[link.target] = (total_weight, current_node)
                    
                    if link.target not in to_visit:
                        to_visit[link.target] = True
        return tree
    
    def get_rev_tree(self, start_node):
        tree = self.get_tree(start_node)
        rtree = {}
        for other_node in tree:
            ptr = rtree
            if other_node == start_node:
                continue
            _, path = spf.get_path(tree, start_node, other_node)
            for n in path:
                if n not in ptr:
                    ptr[n] = {}
                ptr = ptr[n]
        return rtree
    
    def get_path(self, tree, start_node, target_node):
        if target_node not in tree:
            return None
        
        path = []
        current_node = target_node
        while current_node != start_node:
            path.append(current_node)
            current_node = tree[current_node][DijkstraSPF._NODE]
        path.append(start_node)
        
        return tree[target_node][DijkstraSPF._WEIGHT], path[::-1]
    
    def shortest_path(self, start_node, target_node):
        tree = self.get_tree(start_node)
        return self.get_path(tree, start_node, target_node)

In [4]:
# Calculate SPF for network
spf = DijkstraSPF(net)
print(spf.shortest_path(nodes[6], nodes[7]))

(9, [Node(Home), Node(B), Node(D), Node(F), Node(School)])


In [5]:
for node in nodes:
    tree = spf.get_tree(node)
    print('-- {} --'.format(node.name))
#     pprint(tree)
#     print('---')
    pprint(spf.get_rev_tree(node))
    print('---')
    _, path = spf.shortest_path(node, nodes[7])
    print(' -> '.join(map(str,path)))
#     for other_node in tree:
#         if other_node == node:
#             continue
#         _, path = spf.get_path(tree, node, other_node)
#         print(' -> '.join(map(str,path)))
    print()

-- A --
{Node(A): {Node(D): {Node(B): {}, Node(F): {Node(E): {}, Node(School): {}}},
           Node(Home): {Node(C): {}}}}
---
Node(A) -> Node(D) -> Node(F) -> Node(School)

-- B --
{Node(B): {Node(D): {Node(A): {}, Node(F): {Node(School): {}}},
           Node(E): {},
           Node(Home): {Node(C): {}}}}
---
Node(B) -> Node(D) -> Node(F) -> Node(School)

-- C --
{Node(C): {Node(E): {Node(F): {Node(D): {}}, Node(School): {}},
           Node(Home): {Node(A): {}, Node(B): {}}}}
---
Node(C) -> Node(E) -> Node(School)

-- D --
{Node(D): {Node(A): {},
           Node(B): {Node(Home): {Node(C): {}}},
           Node(F): {Node(E): {}, Node(School): {}}}}
---
Node(D) -> Node(F) -> Node(School)

-- E --
{Node(E): {Node(B): {},
           Node(C): {Node(Home): {}},
           Node(F): {Node(D): {Node(A): {}}},
           Node(School): {}}}
---
Node(E) -> Node(School)

-- F --
{Node(F): {Node(D): {Node(A): {}, Node(B): {Node(Home): {}}},
           Node(E): {Node(C): {}},
           Node(Scho